In [5]:
!pip install requests markdown jupyter


In [17]:
# Import necessary libraries
import os
import json
import requests
import markdown
import logging
from datetime import datetime, timedelta  # Updated import
from typing import Dict, List, Any, Optional
from dataclasses import dataclass

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set your API keys here (uncomment and replace with actual keys)
os.environ["OPENROUTER_API_KEY"] = "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"
os.environ["SERPAPI_API_KEY"] = "ade3baf765b90dc72a0e25b327ed502dce09acec1b1a3e1a507f4561aa2d7a08"
os.environ["NEWSAPI_API_KEY"] = "f3ce874447ff415ca2724ddd57e96bc7"

# API Keys - These should be set in environment variables or a secure config
OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY", "YOUR_OPENROUTER_API_KEY")
SERPAPI_API_KEY = os.environ.get("SERPAPI_API_KEY", "YOUR_SERPAPI_API_KEY")
NEWSAPI_API_KEY = os.environ.get("NEWSAPI_API_KEY", "YOUR_NEWSAPI_API_KEY")

# Define agent personas and prompts
PLANNING_AGENT_PERSONA = """You are a highly skilled digital marketer with 10+ years of experience. You are amazing at planning content around given topics. You are very professional and work methodically with other team members to create stunning blog posts. Your expertise in SEO, market trends analysis, and audience targeting guides your planning process. You always include relevant keywords, optimize for search engines, and focus on creating outlines that will appeal to the target audience."""

CONTENT_WRITER_AGENT_PERSONA = """You are a professional world-class blogger who creates content which can attract thousands of readers. You use all your power to create stunning articles that are engaging, informative, and shareable. You work closely with the planning agent and the editor agent to create the best possible blog post. Your writing is conversational yet authoritative, with a perfect balance of entertaining anecdotes and valuable information. You excel at crafting captivating introductions, seamless transitions between sections, and memorable conclusions."""

EDITOR_AGENT_PERSONA = """You are a professional editor who edits and optimizes content to perfection. You work closely with the other team members to produce content of the highest quality. Your keen eye for detail catches grammatical errors, awkward phrasing, and structural inconsistencies. You ensure posts follow SEO best practices, maintain a consistent voice, and adhere to the brand's style guide. You are also skilled at enhancing readability through proper formatting, compelling subheadings, and strategic use of bullet points or numbered lists."""

# Shared memory for agent communication
@dataclass
class SharedMemory:
    topic: str
    research_data: Dict = None
    blog_plan: Dict = None
    blog_draft: str = None
    final_blog: str = None
    messages: List[Dict] = None
    
    def __post_init__(self):
        if self.messages is None:
            self.messages = []
    
    def add_message(self, sender: str, receiver: str, content: str):
        """Add a message to the shared memory."""
        timestamp = datetime.now().isoformat()
        message = {
            "timestamp": timestamp,
            "sender": sender,
            "receiver": receiver,
            "content": content
        }
        self.messages.append(message)
        return message
    
    def get_messages(self, sender=None, receiver=None) -> List[Dict]:
        """Get all messages or filter by sender/receiver."""
        if sender and receiver:
            return [m for m in self.messages if m["sender"] == sender and m["receiver"] == receiver]
        elif sender:
            return [m for m in self.messages if m["sender"] == sender]
        elif receiver:
            return [m for m in self.messages if m["receiver"] == receiver]
        return self.messages

# Base Agent class
class Agent:
    def __init__(self, name: str, persona: str, shared_memory: SharedMemory):
        self.name = name
        self.persona = persona
        self.shared_memory = shared_memory
    
    def send_message(self, receiver: str, content: str) -> Dict:
        """Send a message to another agent."""
        return self.shared_memory.add_message(self.name, receiver, content)
    
    def get_messages_from(self, sender: str) -> List[Dict]:
        """Get messages from a specific sender."""
        return self.shared_memory.get_messages(sender=sender, receiver=self.name)
    
    def query_llm(self, prompt: str, temperature: float = 0.7, max_tokens: int = 1500) -> str:
        """Query the LLM via OpenRouter API."""
        try:
            headers = {
                "Authorization": f"Bearer {OPENROUTER_API_KEY}",
                "Content-Type": "application/json"
            }
            
            data = {
                "model": "google/gemini-2.0-flash-001",  # Add model name here for setting the LLM 
                "messages": [{"role": "user", "content": prompt}],
                "temperature": temperature,
                "max_tokens": max_tokens
            }
            
            response = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers=headers,
                json=data
            )
            
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                logger.error(f"LLM API error: {response.status_code} - {response.text}")
                return f"Error querying LLM: {response.status_code}"
        
        except Exception as e:
            logger.error(f"Exception in query_llm: {e}")
            return f"Error querying LLM: {str(e)}"

# Planning Agent
class PlanningAgent(Agent):
    def __init__(self, shared_memory: SharedMemory):
        super().__init__("PlanningAgent", PLANNING_AGENT_PERSONA, shared_memory)
    
    def search_web(self, query: str, num_results: int = 5) -> List[Dict]:
        """Search the web using SerpAPI."""
        try:
            params = {
                "engine": "google",
                "q": query,
                "api_key": SERPAPI_API_KEY,
                "num": num_results
            }
            
            response = requests.get("https://serpapi.com/search", params=params)
            
            if response.status_code == 200:
                results = response.json()
                organic_results = results.get("organic_results", [])
                return [{
                    "title": result.get("title", ""),
                    "snippet": result.get("snippet", ""),
                    "link": result.get("link", "")
                } for result in organic_results]
            else:
                logger.error(f"SerpAPI error: {response.status_code} - {response.text}")
                return []
        
        except Exception as e:
            logger.error(f"Exception in search_web: {e}")
            return []
    
    def fetch_news(self, query: str, days: int = 7, num_articles: int = 5) -> List[Dict]:
        """Fetch news articles using NewsAPI."""
        try:
            url = "https://newsapi.org/v2/everything"
            
            # Calculate date range (last n days)
            from_date = (datetime.now() - timedelta(days=days)).strftime("%Y-%m-%d")
            
            params = {
                "q": f"{query} recent results",
                "from": from_date,
                "sortBy": "relevancy",
                "apiKey": NEWSAPI_API_KEY,
                "pageSize": num_articles
            }
            
            response = requests.get(url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                articles = data.get("articles", [])
                return [{
                    "title": article.get("title", ""),
                    "description": article.get("description", ""),
                    "url": article.get("url", ""),
                    "source": article.get("source", {}).get("name", ""),
                    "published_at": article.get("publishedAt", "")
                } for article in articles]
            else:
                logger.error(f"NewsAPI error: {response.status_code} - {response.text}")
                return []
        
        except Exception as e:
            logger.error(f"Exception in fetch_news: {e}")
            return []
    
    def conduct_research(self) -> Dict:
        """Conduct research using search and news APIs."""
        topic = self.shared_memory.topic
        logger.info(f"Conducting research on topic: {topic}")
        
        # Search web
        search_results = self.search_web(topic, num_results=7)
        
        # Fetch news
        news_articles = self.fetch_news(topic, days=10, num_articles=8)
        
        research_data = {
            "topic": topic,
            "search_results": search_results,
            "news_articles": news_articles,
            "timestamp": datetime.now().isoformat()
        }
        
        self.shared_memory.research_data = research_data
        return research_data
    
    def create_blog_plan(self) -> Dict:
        """Create a detailed blog post plan based on research."""
        if not self.shared_memory.research_data:
            self.conduct_research()
        
        research_data = self.shared_memory.research_data
        topic = research_data["topic"]
        
        # Format research data for the prompt
        search_results_text = "\n".join([
            f"- {result['title']}: {result['snippet']} (URL: {result['link']})"
            for result in research_data["search_results"]
        ])
        
        news_articles_text = "\n".join([
            f"- {article['title']}: {article['description']} (Source: {article['source']}, URL: {article['url']})"
            for article in research_data["news_articles"]
        ])
        
        # Create prompt for the LLM
        prompt = f"""
        {self.persona}

        You are planning a blog post about "{topic}".

        Here's the research data:

        WEB SEARCH RESULTS:
        {search_results_text}

        RECENT NEWS ARTICLES:
        {news_articles_text}

        Based on this research, create a comprehensive blog post plan with the following elements:
        1. A catchy, SEO-friendly title
        2. Target audience description
        3. Primary keywords (5-7 keywords)
        4. A compelling introduction approach
        5. Detailed outline with main sections and subsections
        6. Key points to cover in each section
        7. Types of examples, data, or case studies to include
        8. A conclusion strategy
        9. Potential sources to cite
        10. Estimated word count
        11. Ensure that the blog post reflects the most recent information available as of March 2025

        Format your response as a structured JSON object with these categories. Be specific and detailed in your plan.
        """
        
        # Query LLM for the blog plan
        try:
            response = self.query_llm(prompt, temperature=0.7, max_tokens=2000)
            
            # Extract JSON from the response
            import re
            json_match = re.search(r'```json\n(.*?)\n```', response, re.DOTALL)
            
            if json_match:
                plan_json = json_match.group(1)
            else:
                # Try to find JSON without the markdown code blocks
                json_match = re.search(r'\{.*\}', response, re.DOTALL)
                if json_match:
                    plan_json = json_match.group(0)
                else:
                    plan_json = response
            
            try:
                blog_plan = json.loads(plan_json)
            except json.JSONDecodeError:
                # If JSON parsing fails, structure the raw text
                blog_plan = {
                    "raw_plan": response,
                    "error": "Could not parse as JSON"
                }
            
            blog_plan["timestamp"] = datetime.now().isoformat()
            self.shared_memory.blog_plan = blog_plan
            
            # Send message to Content Writer Agent
            self.send_message("ContentWriterAgent", f"Blog plan for '{topic}' is ready. You can start writing the draft.")
            
            return blog_plan
        
        except Exception as e:
            logger.error(f"Error creating blog plan: {e}")
            error_plan = {
                "error": str(e),
                "timestamp": datetime.now().isoformat()
            }
            self.shared_memory.blog_plan = error_plan
            return error_plan
    
    def execute_task(self) -> Dict:
        """Execute the planning task."""
        return self.create_blog_plan()

# Content Writer Agent
class ContentWriterAgent(Agent):
    def __init__(self, shared_memory: SharedMemory):
        super().__init__("ContentWriterAgent", CONTENT_WRITER_AGENT_PERSONA, shared_memory)
    
    def check_plan_completeness(self) -> bool:
        """Check if the blog plan has all needed elements."""
        plan = self.shared_memory.blog_plan
        if not plan:
            return False
        
        # Check for minimum required fields
        required_fields = ["title", "outline"]
        for field in required_fields:
            if field not in plan and "raw_plan" not in plan:
                return False
        
        return True
    
    def ask_for_clarification(self, question: str) -> str:
        """Ask Planning Agent for clarification on the blog plan."""
        self.send_message("PlanningAgent", question)
        
        # In a real system, we might wait for a reply here
        # For this demo, we'll immediately query the LLM to simulate a response
        clarification_prompt = f"""
        {PLANNING_AGENT_PERSONA}
        
        You've received the following question from the Content Writer about the blog plan:
        
        "{question}"
        
        Based on the blog plan, provide a helpful clarification. If you don't have enough information, provide the most reasonable guidance based on the original topic.
        """
        
        clarification = self.query_llm(clarification_prompt, temperature=0.7)
        
        # Add the simulated response to shared memory
        self.shared_memory.add_message("PlanningAgent", self.name, clarification)
        
        return clarification
    
    def write_blog_draft(self) -> str:
        """Write the blog draft based on the plan."""
        plan = self.shared_memory.blog_plan
        research_data = self.shared_memory.research_data
        
        if not self.check_plan_completeness():
            clarification = self.ask_for_clarification(
                "Could you provide more details for the blog plan? I need at least a title and outline to proceed."
            )
            # In a real system, we'd wait for a reply and then check again
        
        # Format the blog plan for the prompt
        plan_text = json.dumps(plan, indent=2) if isinstance(plan, dict) else str(plan)
        
        # Format research data
        search_results_text = ""
        news_articles_text = ""
        
        if research_data:
            search_results_text = "\n".join([
                f"- {result['title']}: {result['snippet']} (URL: {result['link']})"
                for result in research_data.get("search_results", [])
            ])
            
            news_articles_text = "\n".join([
                f"- {article['title']}: {article['description']} (Source: {article['source']}, URL: {article['url']})"
                for article in research_data.get("news_articles", [])
            ])
        
        # Create prompt for the LLM
        prompt = f"""
        {self.persona}

        You need to write a high-quality blog post based on the following plan:

        BLOG PLAN:
        {plan_text}

        ADDITIONAL RESEARCH:
        Web Search Results:
        {search_results_text}

        News Articles:
        {news_articles_text}

        Please write a complete blog post that follows all aspects of the plan. Include:
        1. An attention-grabbing introduction
        2. Well-structured sections with clear headings and subheadings
        3. Engaging and informative content that demonstrates expertise
        4. Relevant examples, data points, and citations where appropriate
        5. A compelling conclusion with a call to action
        6. Write the blog post using the provided research data, ensuring all information is up-to-date as of March 2025.
        7. Should be creative in their writing style.

        Format the post in Markdown, with proper use of headings (##, ###), lists, emphasis, and links.
        Aim for a conversational but authoritative tone that builds trust with readers.
        Make sure the content is original, engaging, and provides value to the reader.
        """
        
        # Query LLM for the blog draft
        try:
            blog_draft = self.query_llm(prompt, temperature=0.8, max_tokens=4000)
            self.shared_memory.blog_draft = blog_draft
            
            # Send message to Editor Agent
            self.send_message("EditorAgent", "Blog draft is ready for review and editing.")
            
            return blog_draft
        
        except Exception as e:
            logger.error(f"Error writing blog draft: {e}")
            error_draft = f"Error generating blog draft: {str(e)}"
            self.shared_memory.blog_draft = error_draft
            return error_draft
    
    def execute_task(self) -> str:
        """Execute the content writing task."""
        return self.write_blog_draft()

# Editor Agent
class EditorAgent(Agent):
    def __init__(self, shared_memory: SharedMemory):
        super().__init__("EditorAgent", EDITOR_AGENT_PERSONA, shared_memory)
    
    def request_draft_improvement(self, issue: str) -> str:
        """Request specific improvements from the Content Writer."""
        self.send_message("ContentWriterAgent", issue)
        
        # Simulate response from Content Writer
        improvement_prompt = f"""
        {CONTENT_WRITER_AGENT_PERSONA}
        
        You've received the following editing request:
        
        "{issue}"
        
        Please revise the specific section or aspect of the blog post to address this issue.
        """
        
        improvement = self.query_llm(improvement_prompt, temperature=0.7)
        
        # Add the simulated response to shared memory
        self.shared_memory.add_message("ContentWriterAgent", self.name, improvement)
        
        return improvement
    
    def edit_blog_post(self) -> str:
        """Edit and refine the blog draft."""
        blog_draft = self.shared_memory.blog_draft
        blog_plan = self.shared_memory.blog_plan
        
        if not blog_draft:
            return "No blog draft available to edit."
        
        # Format the blog plan for the prompt
        plan_text = json.dumps(blog_plan, indent=2) if isinstance(blog_plan, dict) else str(blog_plan)
        
        # Create prompt for the LLM
        prompt = f"""
        {self.persona}

        You are editing the following blog post draft:

        ```
        {blog_draft}
        ```

        The post was based on this plan:
        {plan_text}

        Please edit and refine the blog post with these specific goals:
        
        1. Fix any grammar, spelling, punctuation, or style issues
        2. Improve clarity, flow, and readability
        3. Ensure the content follows the original plan
        4. Optimize for SEO (check for keyword usage, meta description potential, etc.)
        5. Add appropriate formatting to enhance readability (headings, bullets, bold/italic text)
        6. Cut any unnecessary content and expand on underdeveloped sections
        7. Ensure the introduction hooks readers and the conclusion provides closure
        8. Verify the post maintains a consistent voice and tone throughout
        
        Return the fully edited blog post in Markdown format. Keep improvements that add value, but maintain the original structure and core content.
        """
        
        # Query LLM for the edited blog post
        try:
            edited_blog = self.query_llm(prompt, temperature=0.6, max_tokens=4000)
            self.shared_memory.final_blog = edited_blog
            
            return edited_blog
        
        except Exception as e:
            logger.error(f"Error editing blog post: {e}")
            error_message = f"Error editing blog post: {str(e)}"
            self.shared_memory.final_blog = blog_draft  # Fallback to unedited draft
            return error_message
    
    def execute_task(self) -> str:
        """Execute the editing task."""
        return self.edit_blog_post()

# Workflow orchestration function
def generate_blog_post(topic: str, output_path: str = None) -> str:
    """
    Orchestrate the entire blog post generation process.
    
    Args:
        topic: The blog post topic
        output_path: Path to save the final blog post (if None, will not save to file)
        
    Returns:
        The final blog post content
    """
    logger.info(f"Starting blog post generation for topic: {topic}")
    
    # Initialize shared memory
    memory = SharedMemory(topic=topic)
    
    # Initialize agents
    planning_agent = PlanningAgent(memory)
    content_writer_agent = ContentWriterAgent(memory)
    editor_agent = EditorAgent(memory)
    
    try:
        # Execute the workflow
        logger.info("Step 1: Research and Planning")
        blog_plan = planning_agent.execute_task()
        logger.info(f"Blog plan created: {blog_plan.get('title', 'Untitled')}")
        
        logger.info("Step 2: Content Writing")
        blog_draft = content_writer_agent.execute_task()
        logger.info(f"Blog draft created ({len(blog_draft)} chars)")
        
        logger.info("Step 3: Editing and Refinement")
        final_blog = editor_agent.execute_task()
        logger.info(f"Final blog post created ({len(final_blog)} chars)")
        
        # Save to file if output_path is provided
        if output_path:
            # Generate a filename based on topic if not specified
            if output_path.endswith('/') or os.path.isdir(output_path):
                title = blog_plan.get("title", "blog_post") if isinstance(blog_plan, dict) else "blog_post"
                filename = "-".join(title.lower().split()[:5]) + ".md"
                full_path = os.path.join(output_path, filename)
            else:
                full_path = output_path
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(os.path.abspath(full_path)), exist_ok=True)
            
            # Write to file
            with open(full_path, 'w', encoding='utf-8') as f:
                f.write(final_blog)
            logger.info(f"Blog post saved to {full_path}")
        
        return final_blog
    
    except Exception as e:
        logger.error(f"Error in blog post generation workflow: {e}")
        return f"Error generating blog post: {str(e)}"

# Example usage
if __name__ == "__main__":
    # Replace with actual topic
    blog_topic = "Sales in the era of AI"
    final_post = generate_blog_post(blog_topic, output_path="./output/")
    print(f"Blog post generated! First 500 chars: {final_post[:500]}...")


2025-03-19 12:40:21,346 - __main__ - INFO - Starting blog post generation for topic: Sales in the era of AI
2025-03-19 12:40:21,349 - __main__ - INFO - Step 1: Research and Planning
2025-03-19 12:40:21,350 - __main__ - INFO - Conducting research on topic: Sales in the era of AI
2025-03-19 12:40:34,970 - __main__ - INFO - Blog plan created: Untitled
2025-03-19 12:40:34,974 - __main__ - INFO - Step 2: Content Writing
2025-03-19 12:40:57,835 - __main__ - INFO - Blog draft created (12585 chars)
2025-03-19 12:40:57,838 - __main__ - INFO - Step 3: Editing and Refinement
2025-03-19 12:41:10,998 - __main__ - INFO - Final blog post created (14700 chars)
2025-03-19 12:41:11,001 - __main__ - INFO - Blog post saved to ./output/blog_post.md


Blog post generated! First 500 chars: ```markdown
## AI in Sales: Revolutionizing Strategies and Driving Results in 2025

Imagine a world where sales leads practically qualify themselves, personalized pitches resonate with every prospect, and sales forecasts are eerily accurate. Sounds like science fiction? Think again. According to a *Harvard Business Review* study, companies leveraging AI for sales have seen a staggering *increase of over 50% in leads, a 60-70% reduction in call time, and significant cost savings* (Alltius.ai). In...
